In [ ]:
''' 
Development Juptyer Notebooks for The Benchmark Calculation Overtime . 
''' 

from abc import abstractmethod, ABC 
import pandas as pd 
from typing import Iterable, List
from datetime import datetime


In [ ]:


''' 
Benchmark Interface/Abstract Class
'''


class Benchmark(ABC):
    @abstractmethod
    def __init__(self, data: pd.DataFrame): 
        self.price_history: pd.Series
        self.df = data 
        self.cur_constituents: List[str] #this is all of current member of the index. 
        self.cur_date: datetime

    def is_reconsistution_date() 

    def 


